In [ ]:
!pip3 install -r requirements.txt

In [ ]:
import os
import random

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
pd.set_option("display.max_colwidth", None)

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from transformers import DistilBertForSequenceClassification, AdamW
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

from utils import *

In [ ]:
df = create_initial_dataframe()

In [ ]:
source_domains = get_source_domains(count=1)
target_domains = get_all_target_domains()

In [ ]:
source_df = df[df["category"].isin(source_domains)]
target_df = df[df["category"].isin(target_domains)]

In [ ]:
# Note: split() function sets seed so as to truly keep target_test unseen until the end  
source_train, source_val = split(source_df)
target_val, target_test = split(target_df)

In [ ]:
# Note: do not touch target_test until final OOS evaluation!
source_train_dataset = create_tokenized_dataset(source_train)
source_val_dataset = create_tokenized_dataset(source_val)
target_val_dataset = create_tokenized_dataset(target_val)

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-cased", num_labels=2)
model.to(device)

In [ ]:
# Constants 
epochs = 3
eps = 1e-8

# Hyperparameters
batch_sizes = [8, 16, 32]
learning_rates = [0.00001, 0.00005, 0.0001, 0.0005, 0.001]